In [15]:
import serial
import struct
import time

##两个imu分开写
state1 = 0
payload1 = list()

state2 = 0
payload2 = list()

# +SPPDATA=1,028,
header1 = bytes([0x2B, 0x53, 0x50, 0x50, 0x44, 0x41, 0x54, 0x41, 0x3D, 0x31, 0x2C, 0x30, 0x32, 0x38, 0x2C])
# +SPPDATA=2,028,
header2= bytes([0x2B, 0x53, 0x50, 0x50, 0x44, 0x41, 0x54, 0x41, 0x3D, 0x32, 0x2C, 0x30, 0x32, 0x38, 0x2C])

class DataFrame():
    def __init__(self, buffer):
        self.device_addr = buffer[0]
        self.data = buffer[1:25]
        self.invariant = buffer[25:28]
        self.reserved = buffer[28:32]

    def __repr__(self):
        vals = struct.unpack("<6f", self.data)
        vals_str = "{:8.4f} {:8.4f} {:8.4f} {:8.4f} {:8.4f} {:8.4f}".format(vals[0], vals[1], vals[2], vals[3], vals[4], vals[5])
        invariant_str = "{:02X} {:02X} {:02X}".format(self.invariant[0], self.invariant[1], self.invariant[2])
        reserved_str = "{:02X} {:02X} {:02X} {:02X}".format(self.reserved[0], self.reserved[1], self.reserved[2], self.reserved[3])
        return "addr:{:02X}|data: {:s}| {:s} | {:s}".format(self.device_addr, vals_str, invariant_str, reserved_str)

def getframe1(b):
    global state1
    global payload1

    b = int.from_bytes(b, byteorder='big')
    if state1 == -1 and b == 0x2B:
        state1 = 0
        return False
    elif state1 >= 0 and state1 < 14:
        if header1[state1 + 1] == b:
            state1 = state1 + 1
        else:
            state1 = -1
        return False
    elif state1 >= 14 and state1 < 45:
        payload1.append(b)
        state1 = state1 + 1
        return False
    elif state1 == 45:
        payload1.append(b)
        state1 = -1
        return True

def getframe2(b):
    global state2
    global payload2

    b = int.from_bytes(b, byteorder='big')
    if state2 == -1 and b == 0x2B:
        state2 = 0
        return False
    elif state2 >= 0 and state2 < 14:
        if header2[state2 + 1] == b:
            state2 = state2 + 1
        else:
            state2 = -1
        return False
    elif state2 >= 14 and state2 < 45:
        payload2.append(b)
        state2 = state2 + 1
        return False
    elif state2 == 45:
        payload2.append(b)
        state2 = -1
        return True

In [16]:
try:
    port_name = "/dev/tty.usbserial-A50285BI"
    port_bps = 921600
    serial_port = serial.Serial(port_name, port_bps, timeout=1)
    print(serial_port)
    serial_port.write("AT+SPPSEND=1,6,view=1\r\n".encode(encoding="ascii"))
    time.sleep(5)
    serial_port.write("AT+SPPSEND=2,6,view=1\r\n".encode(encoding="ascii"))

    print("command sent")
    f1=open('imu1.txt','w')
    
    with open("data.hex", "wb") as f:

        for i in range(10240):
            data = serial_port.read()
            if getframe1(data):
                frame = DataFrame(bytes(payload1))
                print(frame)
                payload1 = []
            if getframe2(data):
                frame = DataFrame(bytes(payload2))
                print(frame)
                payload2=[]
            f.write(data)

    serial_port.close()

except Exception as e:
    print(e)

Serial<id=0x7fd76cedd940, open=True>(port='/dev/tty.usbserial-A50285BI', baudrate=921600, bytesize=8, parity='N', stopbits=1, timeout=1, xonxoff=False, rtscts=False, dsrdtr=False)
command sent
addr:0B|data: 146.3716   1.7667   2.6505  -0.0081  -0.0326  -0.0003| BB 0D 0A | 0D 0A 0D 0A
addr:0B|data: 146.3706   1.7610   2.6774  -0.0071  -0.0333  -0.0032| BB 0D 0A | 0D 0A 0D 0A
addr:0A|data:  -3.0722   2.4995   0.0653   0.0328  -0.0195  -1.0162| BB 0D 0A | 0D 0A 0D 0A
addr:0A|data:  -3.0723   2.4999   0.0654   0.0344  -0.0200  -1.0167| BB 0D 0A | 0D 0A 0D 0A
addr:0A|data:  -3.0714   2.4995   0.0658   0.0343  -0.0193  -1.0173| BB 0D 0A | 0D 0A 0D 0A
addr:0B|data: 146.3638   1.7676   2.6334  -0.0083  -0.0341  -0.0032| BB 0D 0A | 0D 0A 0D 0A
addr:0A|data:  -3.0714   2.4995   0.0658   0.0343  -0.0193  -1.0173| BB 0D 0A | 0D 0A 0D 0A
addr:0A|data:  -3.0714   2.4884   0.0740   0.0334  -0.0197  -1.0179| BB 0D 0A | 0D 0A 0D 0A
addr:0A|data:  -3.0714   2.4884   0.0736   0.0356  -0.0200  -1.0173| BB

In [6]:
port_name = "/dev/tty.usbserial-A50285BI"
port_bps = 921600
serial_port = serial.Serial(port_name, port_bps, timeout=1)

In [7]:
serial_port.write("AT+SPPSEND=1,6,view=1\r\n".encode(encoding="ascii"))

23

In [8]:
serial_port.write("AT+SPPSEND=2,6,view=1\r\n".encode(encoding="ascii"))

23

In [9]:
for i in range(10024):
    data = serial_port.readline()
    print(data)

b'\r\n'
b'+SPPDATA=2,028,\n'
b'\xe0\r\n'
b'+SPPDATA=1,028,\x0beQ\x14CS\xa7\xe0?\xa0d.@\xa3#\x08\xbc\x1c\xba\x0f\xbd\x81\xd60\xbc\xbb\r\n'
b'\r\n'
b'\r\n'
b'+SPPDATA=1,028,\x0b"Q\x14C\x002\xe0?\xd6}0@bY;\xbc){\x0f\xbd`\xddc:\xbb\r\n'
b'\r\n'
b'\r\n'
b'+SPPDATA=2,028,\n'
b'\x10HE\xc0\x99\x9d @\x04\xc3\xb4;\x00\x80\x0f=\x00\x00\xa5\xbc\x00\xc4\x81\xbf\xbb\r\n'
b'\r\n'
b'\r\n'
b'+SPPDATA=2,028,\n'
b"\xc4HE\xc0\r\xac @W'\xb2;\x00\x80\x08=\x00\x00\xa0\xbc\x00\x80\x82\xbf\xbb\r\n"
b'\r\n'
b'\r\n'
b'+SPPDATA=2,028,\n'
b'\x15OE\xc0{\xb4 @C\xa5\xa9;\x00\x00\x0e=\x00\x00\xa7\xbc\x00\xc0\x83\xbf\xbb\r\n'
b'\r\n'
b'\r\n'
b'+SPPDATA=1,028,\x0b\x96P\x14C\x11\xa9\xe1?\x9c\x0c.@\xee\xfb\t\xbc\xbaf\n'
b'\xbd\xc1\x04]<\xbb\r\n'
b'\r\n'
b'\r\n'
b'+SPPDATA=2,028,\n'
b'!UE\xc0Q\xae @\x84d\xa1;\x00\x80\x10=\x00\x00\xa0\xbc\x00H\x83\xbf\xbb\r\n'
b'\r\n'
b'\r\n'
b'+SPPDATA=2,028,\n'
b'm\\E\xc0o\xeb @\x8d\xf2L<\x00\x00\x11=\x00\x00\xa7\xbc\x00\x8c\x82\xbf\xbb\r\n'
b'\r\n'
b'\r\n'
b'+SPPDATA=2,028,\n'
b'm\\E\xc0

b'+SPPDATA=2,028,\n'
b'\x01 E\xc0\x1c\xf5 @\xc8.\r=\x00\x00\x07=\x00\x00\x9b\xbc\x00\x88\x82\xbf\xbb\r\n'
b'\r\n'
b'\r\n'
b'+SPPDATA=2,028,\n'
b'\xf2\x1eE\xc0\xb2\xf8 @\xee&\r=\x00\x00\t=\x00\x00\x97\xbc\x00l\x82\xbf\xbb\r\n'
b'\r\n'
b'\r\n'
b'+SPPDATA=2,028,\n'
b'\x95\x0fE\xc0\x17\xca @\xae\xdf\x91=\x00\x00\n'
b'=\x00\x00\x9f\xbc\x00H\x82\xbf\xbb\r\n'
b'\r\n'
b'\r\n'
b'+SPPDATA=2,028,\n'
b'x\x1cE\xc0\x8f\xc2 @K\xc9\x90=\x00\x00\x03=\x00\x00\xa6\xbc\x00\xcc\x81\xbf\xbb\r\n'
b'\r\n'
b'\r\n'
b'+SPPDATA=2,028,\n'
b'\xa7 E\xc0\xca\xc0 @U)\x91=\x00\x00\x0b=\x00\x00\x9d\xbc\x00\xc0\x81\xbf\xbb\r\n'
b'\r\n'
b'\r\n'
b'+SPPDATA=2,028,\n'
b'T\x1cE\xc0\xc6\xc1 @\xea\xed\x91=\x00\x00\x0b=\x00\x00\xa0\xbc\x00\xc8\x81\xbf\xbb\r\n'
b'\r\n'
b'\r\n'
b'+SPPDATA=2,028,\n'
b'T\x1cE\xc0\xc6\xc1 @\xea\xed\x91=\x00\x00\x0b=\x00\x00\xa0\xbc\x00\xc8\x81\xbf\xbb\r\n'
b'\r\n'
b'\r\n'
b'+SPPDATA=2,028,\n'
b'\x8f\rE\xc0\xd7\xaa @\xa5r\x8e=\x00\x80\x08=\x00\x00\x9d\xbc\x00\x88\x82\xbf\xbb\r\n'
b'\r\n'
b'\r\n'
b'+SP

b'd*E\xc0\xd1/"@!:\x98=\x00\x80\r=\x00\x00\xb7\xbc\x00D\x82\xbf\xbb\r\n'
b'\r\n'
b'\r\n'
b'+SPPDATA=2,028,\n'
b'\x96.E\xc0!("@A}\x97=\x00\x00\x0b=\x00\x00\xa2\xbc\x00 }\xbf\xbb\r\n'
b'\r\n'
b'\r\n'
b'+SPPDATA=2,028,\n'
b'\x8f*E\xc0\xfa<"@\x12*\x97=\x00\x00\t=\x00\x00\xab\xbc\x00\xf0\x86\xbf\xbb\r\n'
b'\r\n'
b'\r\n'
b'+SPPDATA=2,028,\n'
b'\x824E\xc0!A"@l\x8e\x99=\x00\x00\x19=\x00\x00\x9a\xbc\x00\xf4\x80\xbf\xbb\r\n'
b'\r\n'
b'\r\n'
b'+SPPDATA=2,028,\n'
b'\xa0 E\xc0\xf8\x12"@\xe5\x92\xab=\x00\x80\x1e=\x00\x00\x9d\xbc\x00@\x83\xbf\xbb\r\n'
b'\r\n'
b'\r\n'
b'+SPPDATA=2,028,\n'
b'\xa4(E\xc0`\x05"@\xb86\xac=\x00\x80\n'
b'=\x00\x00\xa8\xbc\x00\x84\x82\xbf\xbb\r\n'
b'\r\n'
b'\r\n'
b'+SPPDATA=1,028,\x0b\xd3\n'
b'\x14C\xd4\x1f\xde?\xb8K4@\x1a\x15i\xbc\xc1\xb6\x14\xbd\xc0\xc0$;\xbb\r\n'
b'\r\n'
b'\r\n'
b'+SPPDATA=1,028,\x0b\xa2\n'
b'\x14C_\x13\xe0?\xc2\x1c1@\x0ez\n'
b'\xbc\x9f~\x11\xbd\xc2\x93\x1c\xbb\xbb\r\n'
b'\r\n'
b'\r\n'
b'+SPPDATA=2,028,\n'
b'\xa4(E\xc0`\x05"@\xb86\xac=\x00\x80\n'
b'=\x00\x

b'+SPPDATA=1,028,\x0b\x7f\xfd\x13C\xed|\xdf?Mp1@\x08\xdf\x12\xbc\xb0%\x10\xbd\x01og\xbb\xbb\r\n'
b'\r\n'
b'\r\n'
b'+SPPDATA=1,028,\x0b\x11\xfd\x13C@\xb7\xde?\xaf+3@\xb4\xa5\x16\xbc\r\x9d\x10\xbd\x02\x14\x0b:\xbb\r\n'
b'\r\n'
b'\r\n'
b'+SPPDATA=2,028,\n'
b'\xf6\x16E\xc0\xb3\xd0"@\x9d\x13\xb9=\x00\x80\x0c=\x00\x00\xa1\xbc\x00\xdc\x81\xbf\xbb\r\n'
b'\r\n'
b'\r\n'
b'+SPPDATA=2,028,\n'
b'\x9a\x1fE\xc0\x85\xe0"@\xa6\xf4\xb8=\x00\x80\x11=\x00\x00\xa4\xbc\x00l\x82\xbf\xbb\r\n'
b'\r\n'
b'\r\n'
b'+SPPDATA=2,028,\n'
b'S\x1bE\xc0\x0e\xe5"@\xd4n\xb8=\x00\x00\x0b=\x00\x00\x9c\xbc\x00p\x82\xbf\xbb\r\n'
b'\r\n'
b'\r\n'
b'+SPPDATA=1,028,\x0b\x90\xfd\x13C\xe0\x96\xe0?8\xd10@\x8al\xc9\xbb@\\\x0e\xbd\xe0?.7\xbb\r\n'
b'\r\n'
b'\r\n'
b'+SPPDATA=1,028,\x0b8\xfd\x13C\x03\xa2\xdf?\xf7\x892@\xad\xa7\x14\xbc\xe8\xe1\x0e\xbd\x03Kf\xbb\xbb\r\n'
b'\r\n'
b'\r\n'
b'+SPPDATA=2,028,\n'
b'\x97\x1cE\xc0[\xd8"@dB\xb8=\x00\x00\x08=\x00\x00\xa0\xbc\x00\x00\x82\xbf\xbb\r\n'
b'\r\n'
b'\r\n'
b'+SPPDATA=2,028,\n'
b'\xe8\x1aE\xc

b'+SPPDATA=1,028,\x0bQ\xf0\x13C\x7f\xaf\xe1?\xec\xe82@l\xdd!\xbc\xf0\xd5\x10\xbd\x80<M\xbb\xbb\r\n'
b'\r\n'
b'\r\n'
b'+SPPDATA=1,028,\x0b\xe6\xf0\x13C\x19\xf0\xe3?\xc9R0@\x0e\xe4\x08\xbc(\xec\x0f\xbd\x04\xa2f:\xbb\r\n'
b'\r\n'
b'\r\n'
b'+SPPDATA=2,028,\n'
b'\x1eRE\xc0\xf6O!@!\x17\x8a=\x00\x00\n'
b'=\x00\x00\xa3\xbc\x00\xd8\x81\xbf\xbb\r\n'
b'\r\n'
b'\r\n'
b'+SPPDATA=2,028,\n'
b'\x1eRE\xc0\xf6O!@!\x17\x8a=\x00\x00\n'
b'=\x00\x00\xa3\xbc\x00\xd8\x81\xbf\xbb\r\n'
b'\r\n'
b'\r\n'
b'+SPPDATA=2,028,\n'
b'\xfdYE\xc0:]!@\xe1\x9e\x8b=\x00\x80\x0c=\x00\x00\xa3\xbc\x00d\x82\xbf\xbb\r\n'
b'\r\n'
b'\r\n'
b'+SPPDATA=2,028,\n'
b'\xb1ZE\xc0\xea\xee!@\xba\xa3\x87=\x00\x00\x0c=\x00\x00\xa5\xbc\x004\x82\xbf\xbb\r\n'
b'\r\n'
b'\r\n'
b'+SPPDATA=1,028,\x0b\xcb\xf0\x13Cvl\xe3?\x9e:1@\x8c\xdc\x11\xbc\xc4u\x11\xbd\x04\xa0]\xb9\xbb\r\n'
b'\r\n'
b'\r\n'
b'+SPPDATA=1,028,\x0b\xb6\xf0\x13C\xefc\xe2?\xd8\x043@ \x0c\x02\xbc\x96m\x19\xbd\xc2u\x15\xbb\xbb\r\n'
b'\r\n'
b'\r\n'
b'+SPPDATA=2,028,\n'
b'{[E\xc0\x1d\xe1!@\x

...

b'+SPPDATA=1,028,\x0bO\xb0\x12C\x80+\xe2?\x16\xf6.@\xa1\xd2\x1a\xbc\x9c\xe7\x0b\xbd@~@\xbb\xbb\r\n'
b'\r\n'
b'\r\n'
b'+SPPDATA=1,028,\x0b\xa8\xae\x12C[\xc6\xe2?\xfe\xe7,@d[\xf7\xbbe\x87\x0c\xbd\x84\\\n'
b'\xbb\xbb\r\n'
b'\r\n'
b'\r\n'
b'+SPPDATA=2,028,\n'
b'\n'
b'\xe7D\xc0\x92j#@\xef\x95X=\x00\x00\x11=\x00\x00\x96\xbc\x00\xf4\x81\xbf\xbb\r\n'
b'\r\n'
b'\r\n'
b'+SPPDATA=2,028,\n'
b'C\xe5D\xc0\xccl#@\xed\tY=\x00\x00\x0b=\x00\x00\xa0\xbc\x00<\x82\xbf\xbb\r\n'
b'\r\n'
b'\r\n'
b'+SPPDATA=2,028,\n'
b'\xe8\xe0D\xc0\x03P$@\xa80>=\x00\x80\x13=\x00\x00\xa4\xbc\x00H\x82\xbf\xbb\r\n'
b'\r\n'
b'\r\n'
b'+SPPDATA=2,028,\n'
b'\xe8\xe0D\xc0\x03P$@\xa80>=\x00\x80\x13=\x00\x00\xa4\xbc\x00H\x82\xbf\xbb\r\n'
b'\r\n'
b'\r\n'
b"+SPPDATA=1,028,\x0b}\xae\x12C\x02\xd0\xe1?\x0e\xb2.@'\x86\x17\xbc1\x97\x0e\xbd\x00p\xf28\xbb\r\n"
b'\r\n'
b'\r\n'
b'+SPPDATA=1,028,\x0bA\xad\x12C[\xe3\xe2?@\x9e+@\xf0\xb0\x02\xbc\xa1/\r\xbd~\xceX\xbb\xbb\r\n'
b'\r\n'
b'\r\n'
b'+SPPDATA=2,028,\n'
b'\x89\xddD\xc0ET$@\xadj@=\x00\x00\x15=

b'\r\n'
b'+SPPDATA=2,028,\n'
b'D\xbfD\xc0\xe3\x9c$@GCG=\x00\x80\x0c=\x00\x00\xa5\xbc\x00@\x82\xbf\xbb\r\n'
b'\r\n'
b'\r\n'
b'+SPPDATA=2,028,\n'
b'\xa5\xb7D\xc0\x02\xa1$@(+G=\x00\x00\x14=\x00\x00\x9d\xbc\x00\x08\x82\xbf\xbb\r\n'
b'\r\n'
b'\r\n'
b'+SPPDATA=2,028,\n'
b'\x14\xbeD\xc0\xd4\x9c$@\x07SH=\x00\x00\x0b=\x00\x00\x9f\xbc\x00\x18\x82\xbf\xbb\r\n'
b'\r\n'
b'\r\n'
b'+SPPDATA=2,028,\n'
b'\x1d\xb9D\xc0\x92\x95$@\x0fPJ=\x00\x80\x0c=\x00\x00\xa0\xbc\x004\x82\xbf\xbb\r\n'
b'\r\n'
b'\r\n'
b'+SPPDATA=1,028,\x0b\x8e\x9e\x12C4X\xe2?\r\xae-@\xd6\xf9\r\xbc\x91\x02\x16\xbd\x02\x10\x1a\xbb\xbb\r\n'
b'\r\n'
b'\r\n'
b'+SPPDATA=1,028,\x0b\x8e\x9e\x12C\xc1\x1e\xe4?\xb5\xe4*@\x14\xb0 \xbc\x109\x0c\xbd\xc1\xe3\x18\xbb\xbb\r\n'
b'\r\n'
b'\r\n'
b'+SPPDATA=1,028,\x0b6\x9e\x12C\tG\xe3?\xc8\xb9,@\xca]\x0e\xbc4d\n'
b'\xbd\x00\xf6\x06\xba\xbb\r\n'
b'\r\n'
b'\r\n'
b'+SPPDATA=2,028,\n'
b'2\xb7D\xc0\xdf\x9c$@\xc1\x13J=\x00\x80\x13=\x00\x00\xa6\xbc\x00h\x82\xbf\xbb\r\n'
b'\r\n'
b'\r\n'
b'+SPPDATA=2,028,\n'
b'2\xb7

b'\r\n'
b'+SPPDATA=1,028,\x0b\x86\x8b\x12C}\x88\xdf?W\x8b-@V\x85 \xbc\xecU\x0f\xbd\x00\x12i\xba\xbb\r\n'
b'\r\n'
b'\r\n'
b'+SPPDATA=1,028,\x0b\xde\x89\x12C*\x97\xe0?N{*@\x16\x98\t\xbc\xdad\x0c\xbd\x00\x0c\xb4\xba\xbb\r\n'
b'\r\n'
b'\r\n'
b'+SPPDATA=2,028,\n'
b'7\xe6D\xc0\xa2\xc0$@0\xea^=\x00\x00\x0c=\x00\x00\x99\xbc\x00\x08\x82\xbf\xbb\r\n'
b'\r\n'
b'\r\n'
b'+SPPDATA=2,028,\n'
b'\xc3\xcaD\xc0v\xc6$@$\xcf\x9b=\x00\x00\n'
b'=\x00\x00\xab\xbc\x00\x04\x82\xbf\xbb\r\n'
b'\r\n'
b'\r\n'
b'+SPPDATA=2,028,\n'
b'\x1c\xc6D\xc0\x9b\xc2$@\x8a_\x9c=\x00\x00\x10=\x00\x00\xa0\xbc\x00H\x82\xbf\xbb\r\n'
b'\r\n'
b'\r\n'
b'+SPPDATA=2,028,\n'
b'\xf1\xccD\xc0\xeb\xc4$@\xb8\xda\x9c=\x00\x80\x0e=\x00\x00\xa4\xbc\x00\x1c\x82\xbf\xbb\r\n'
b'\r\n'
b'\r\n'
b'+SPPDATA=1,028,\x0b\xa9\x89\x12C\xa3\xcf\xdf?\x97P,@v,\x16\xbc\xf8n\t\xbd\xc2\x96c\xbb\xbb\r\n'
b'\r\n'
b'\r\n'
b'+SPPDATA=1,028,\x0br\x89\x12C,\xeb\xde?\xea%.@x\x19\n'
b'\xbc^p\x0e\xbd\x02o2\xbb\xbb\r\n'
b'\r\n'
b'\r\n'
b'+SPPDATA=1,028,\x0b\xba\x89\x12C\xfb

b'+SPPDATA=1,028,\x0b\x07|\x12C\xba\x91\xe1?\x808(@\x9b^\x02\xbcT\xc8\x05\xbd\x02;\x14\xbb\xbb\r\n'
b'\r\n'
b'\r\n'
b'+SPPDATA=2,028,\n'
b'\xfd\xe9D\xc0\x1d\x1f!@\x10\xe5w=\x00\x00\x0b=\x00\x00\x95\xbc\x00\x1c\x82\xbf\xbb\r\n'
b'\r\n'
b'\r\n'
b'+SPPDATA=2,028,\n'
b'\x1a\xf2D\xc0\x12"!@\xe7\xccy=\x00\x80\n'
b'=\x00\x00\xa0\xbc\x00$\x82\xbf\xbb\r\n'
b'\r\n'
b'\r\n'
b'+SPPDATA=1,028,\x0b\xaf{\x12C\x1c\x97\xe0?o\xff)@\x11]\x08\xbc\xb1\xd4\x07\xbd\xc20\r\xbb\xbb\r\n'
b'\r\n'
b'\r\n'
b'+SPPDATA=1,028,\x0br{\x12C\xdd\xe0\xdf?8\xa7+@\xca"\x0e\xbcC\xb5\x07\xbd\x03q\xb7\xba\xbb\r\n'
b'\r\n'
b'\r\n'
b'+SPPDATA=2,028,\n'
b'\xc2\xebD\xc0\x06\xd7 @<\xfcX=\x00\x00\x08=\x00\x00\x99\xbc\x00\x0c\x82\xbf\xbb\r\n'
b'\r\n'
b'\r\n'
b'+SPPDATA=2,028,\n'
b'\x07\xebD\xc0\x93\xcd @j\x0fX=\x00\x00\x0f=\x00\x00\xa4\xbc\x00\x18\x82\xbf\xbb\r\n'
b'\r\n'
b'\r\n'
b'+SPPDATA=1,028,\x0b\x13{\x12C><\xe1?\x9b\x04)@\x9a\xe8$\xbc\xbe\xc7\x01\xbd\x80t\xea\xba\xbb\r\n'
b'\r\n'
b'\r\n'
b'+SPPDATA=1,028,\x0b\xb0z\x12C\x9cj\xe0

b'P\xc5D\xc0\xe8,"@\x02J\xaa=\x00\x00\x10=\x00\x00\xa4\xbc\x000\x82\xbf\xbb\r\n'
b'\r\n'
b'\r\n'
b'+SPPDATA=1,028,\x0b\xc6p\x12C\xfe\xd2\xe0?\xd2b-@\xba\xce\x07\xbc7\xa0\x0f\xbd\x82\xc4\xc4\xba\xbb\r\n'
b'\r\n'
b'\r\n'
b'+SPPDATA=1,028,\x0bNo\x12C1\xab\xe1?\xf6\xb4*@\x1a\x01\x16\xbc\xf8*\x07\xbd\x02\xd3\xa7\xba\xbb\r\n'
b'\r\n'
b'\r\n'
b'+SPPDATA=1,028,\x0b\x04o\x12Ca\xde\xe0?\xb7m,@1\xd1\x05\xbc.\x8b\n'
b'\xbd\x05yk\xba\xbb\r\n'
b'\r\n'
b'\r\n'
b'+SPPDATA=2,028,\n'
b'\xc8\xbaD\xc03\xf3!@\x17y\xb5=\x00\x00\r=\x00\x00\xa8\xbc\x00X\x82\xbf\xbb\r\n'
b'\r\n'
b'\r\n'
b'+SPPDATA=2,028,\n'
b'\xc8\xbaD\xc03\xf3!@\x17y\xb5=\x00\x00\r=\x00\x00\xa8\xbc\x00X\x82\xbf\xbb\r\n'
b'\r\n'
b'\r\n'
b'+SPPDATA=2,028,\n'
b'\xba\xb7D\xc0\x84\xf8!@]\x1c\xb6=\x00\x80\x0b=\x00\x00\x9e\xbc\x00P\x82\xbf\xbb\r\n'
b'\r\n'
b'\r\n'
b'+SPPDATA=2,028,\n'
b' \xbdD\xc01\xf8!@B\xfd\xb5=\x00\x80\x13=\x00\x00\xa4\xbc\x00\x08\x82\xbf\xbb\r\n'
b'\r\n'
b'\r\n'
b'+SPPDATA=2,028,\n'
b'\xd9\xbcD\xc0\xbb\xf7!@|\xf6\xb6=\x00\x80\r=

In [14]:
header

b'+SPPDATA=1,028,'

In [13]:
header2

b'+SPPDATA=2,028,'